# IND320 - Assignment 4

Name: Eskil Torp Skogsholm 

GitHub link: 

Streamlit app: 


## AI usage


## Log


## Tasks

In [62]:
import pandas as pd
import requests
from datetime import datetime
import calendar

def get_elhub_data(year_start, year_end, dataset):

    base_url = "https://elhub.no/api/v1/measurements"
    entity = "price-areas"
    urls = []
    all_data = []

    for year in range(year_start, year_end + 1):

        monthly_ranges = []
        for month in range(1, 13):
            # First day of the month
            start = datetime(year, month, 1, 0, 0, 0)
            
            # Last day of the month
            last_day = calendar.monthrange(year, month)[1]
            end = datetime(year, month, last_day, 23, 59, 59)
            
            # Format as ISO8601 with URL-encoded +01
            start_str = start.strftime("%Y-%m-%dT%H:%M:%S") + "%2B01"
            end_str = end.strftime("%Y-%m-%dT%H:%M:%S") + "%2B01"
            
            monthly_ranges.append((start_str, end_str))
        
    
        for month_start, month_end in monthly_ranges:
            url = f"https://api.elhub.no/energy-data/v0/{entity}?dataset={dataset}&startDate={month_start}&endDate={month_end}"
            urls.append(url)
    
    for url in urls:
        try:
            response = requests.get(url)
            data = response.json()

            print(f"Fetched data from {url}")

            for entry in data.get("data", []):
                attrs = entry.get("attributes", {})

                if "productionPerGroupMbaHour" in attrs:
                    monthly_list = attrs["productionPerGroupMbaHour"]
                elif "consumptionPerGroupMbaHour" in attrs:
                    monthly_list = attrs["consumptionPerGroupMbaHour"]
                else:
                    monthly_list = []
                
                all_data.extend(monthly_list)
        except Exception as e:
            print(f"Error fetching data from {url}: {e}")
        
    
    return pd.DataFrame(all_data)
    

  


In [63]:
test_df = get_elhub_data(2021, 2021,  "PRODUCTION_PER_GROUP_MBA_HOUR")

Fetched data from https://api.elhub.no/energy-data/v0/price-areas?dataset=PRODUCTION_PER_GROUP_MBA_HOUR&startDate=2021-01-01T00:00:00%2B01&endDate=2021-01-31T23:59:59%2B01
Fetched data from https://api.elhub.no/energy-data/v0/price-areas?dataset=PRODUCTION_PER_GROUP_MBA_HOUR&startDate=2021-02-01T00:00:00%2B01&endDate=2021-02-28T23:59:59%2B01
Fetched data from https://api.elhub.no/energy-data/v0/price-areas?dataset=PRODUCTION_PER_GROUP_MBA_HOUR&startDate=2021-03-01T00:00:00%2B01&endDate=2021-03-31T23:59:59%2B01
Fetched data from https://api.elhub.no/energy-data/v0/price-areas?dataset=PRODUCTION_PER_GROUP_MBA_HOUR&startDate=2021-04-01T00:00:00%2B01&endDate=2021-04-30T23:59:59%2B01
Fetched data from https://api.elhub.no/energy-data/v0/price-areas?dataset=PRODUCTION_PER_GROUP_MBA_HOUR&startDate=2021-05-01T00:00:00%2B01&endDate=2021-05-31T23:59:59%2B01
Fetched data from https://api.elhub.no/energy-data/v0/price-areas?dataset=PRODUCTION_PER_GROUP_MBA_HOUR&startDate=2021-06-01T00:00:00%2B01&e

In [64]:
test_df.head()

,endTime,lastUpdatedTime,priceArea,productionGroup,quantityKwh,startTime
0,2021-01-01T01:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2507716.8,2021-01-01T00:00:00+01:00
1,2021-01-01T02:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2494728.0,2021-01-01T01:00:00+01:00
2,2021-01-01T03:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2486777.5,2021-01-01T02:00:00+01:00
3,2021-01-01T04:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2461176.0,2021-01-01T03:00:00+01:00
4,2021-01-01T05:00:00+01:00,2024-12-20T10:35:40+01:00,NO1,hydro,2466969.2,2021-01-01T04:00:00+01:00
